In [17]:
# Creates new business-level conceptual entities into the DB from an excel sheet
# Yes that's overkill for single records
import sqlite3
import pandas as pd
import sys
sys.path.append('../')
from user_packages import hashing

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

TargetExcelFilename = 'business.xlsx'
TargetExcelFilepath = 'imports/business/'

In [18]:
df = pd.read_excel(TargetExcelFilepath + TargetExcelFilename)
df.fillna('', inplace=True)

df['ModelBusinessKeyPhrase'] = df.apply(lambda row: row['ModelKeyPhrase'] + ':' + row['BusinessName'], axis=1)

df

,ModelKeyPhrase,BusinessName,ModelBusinessKeyPhrase
0,yesterday.south.pupil,Openwork,yesterday.south.pupil:Openwork
1,yesterday.south.pupil,ZenithOne,yesterday.south.pupil:ZenithOne
2,yesterday.south.pupil,Intelliflo Office,yesterday.south.pupil:Intelliflo Office


In [19]:
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['ModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessHashKey'
  , columns = ['BusinessName']
)
# Commented as there are no satellite columns
#df = hashing.add_md5_hash_column(
#  df
#  , md5_column_name = 'HashDiff'
#  , columns = ['BusinessName']
#)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelBusinessHashKey'
  , columns = ['ModelBusinessKeyPhrase']
)
df['RecordSource'] = df.apply(lambda row: 'Py.CreateBusiness', axis=1)

df

,ModelKeyPhrase,BusinessName,ModelBusinessKeyPhrase,ModelHashKey,BusinessHashKey,ModelBusinessHashKey,RecordSource
0,yesterday.south.pupil,Openwork,yesterday.south.pupil:Openwork,d3b890871dec51c04fd3e6e2d63f7282,e52141e4f34538b9d385ba696d57673e,20ee37c2757a201e32da2625639d5ed8,Py.CreateBusiness
1,yesterday.south.pupil,ZenithOne,yesterday.south.pupil:ZenithOne,d3b890871dec51c04fd3e6e2d63f7282,9cc5c8771b6616394bd02eef81a685f2,0fed41afe362ee2404945d851b70f23b,Py.CreateBusiness
2,yesterday.south.pupil,Intelliflo Office,yesterday.south.pupil:Intelliflo Office,d3b890871dec51c04fd3e6e2d63f7282,7710a92417f1bf7fd4a38a6eb16502e3,55b0e19fda1d5caf070dd00b65ee0592,Py.CreateBusiness


In [20]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_ExcelToBusiness")
conn.commit()
df.to_sql('stg_Py_ExcelToBusiness', conn, if_exists='append', index=False)

3

In [22]:
# Write to the Hub (Business)
sql_query = """
INSERT INTO rv_h_Business
(
    BusinessHashKey
  , LoadDate
  , RecordSource
  , BusinessName
)
SELECT DISTINCT
    BusinessHashKey
  , LoadDate
  , RecordSource
  , BusinessName
FROM
  stg_Py_ExcelToBusiness
WHERE
  BusinessHashKey NOT IN (SELECT BusinessHashKey FROM rv_h_Business)
""";
conn.execute(sql_query)
conn.commit()

In [23]:
# Write to the Link (Model - Business)
sql_query = """
INSERT INTO rv_l_ModelBusiness
(
  ModelBusinessHashKey
  , LoadDate
  , RecordSource
  , ModelHashKey
  , BusinessHashKey
)
SELECT
  stg.ModelBusinessHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ModelHashKey
  , stg.BusinessHashKey
FROM
  stg_Py_ExcelToBusiness stg
WHERE
  stg.ModelBusinessHashKey IS NOT NULL
  AND ModelBusinessHashKey NOT IN (
    SELECT ModelBusinessHashKey
    FROM rv_l_ModelBusiness
  )
""";
conn.execute(sql_query)
conn.commit()